CNN으로 autoencoder 구현

In [1]:
%matplotlib inline
from keras import layers, models

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


autoencoder CNN 모델

In [2]:
def Conv2D(filters, kernel_size, padding='same', activation='relu'):
    return layers.Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation=activation)

class AE(models.Model):
    def __init__(self, original_shape=(1, 28, 28)):
        # input
        original = layers.Input(shape=original_shape)
        
        ######################
        # encoder :
        ######################
        
        # encoding 1 : conv + maxpool
        x = Conv2D(filters=4, kernel_size=(3,3))(original)
        x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        
        # encoding 2 : filter수만 늘어나고 동일
        x = Conv2D(8, (3,3))(x)
        x = layers.MaxPooling2D((2,2), padding='same')(x)
        
        # encoding 3 : encoding 끝. 7x7의 2차원 이미지 출력
        z = Conv2D(1, (7,7))(x)
        
        ######################
        # decoder :
        ######################
        
        '''
        # 아래 링크에서 UpSampling 관련 내용 확인할 수 있음
        https://kharshit.github.io/blog/2019/02/15/autoencoder-downsampling-and-upsampling
        '''
        
        # decoding 1 : 
        y = Conv2D(16, (3,3))(z)
        y = layers.UpSampling2D((2,2))(y) #width/height가 2배로 증가
        
        # decoding 2 : 
        y = Conv2D(8, (3,3))(y)
        y = layers.UpSampling2D((2,2))(y)
        
        # decoding 3:
        y = Conv2D(4, (3,3))(y)
        
        """
        # 원본으로 복원 과정
        channel을 1로 줄이고 sigmoid 처리를 해 준다.
        """
        decoded = Conv2D(1, (3,3), activation='sigmoid')(y)
        
        super().__init__(original, decoded) #input & output
        self.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

MNIST 데이터 준비
픽셀값을 0~1 사이로 변경하고 label을 one-hot encoding하지만 

In [3]:
from keras import datasets 
import keras 
from random import randint
from keras.preprocessing.image import ImageDataGenerator


class DATA():
    def __init__(self):
        num_classes = 10

        (x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()
        img_rows, img_cols = x_train.shape[1:]

        if backend.image_data_format() == 'channels_first':
            x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
            x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
            input_shape = (1, img_rows, img_cols)
        else:
            x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
            x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
            input_shape = (img_rows, img_cols, 1)

        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train /= 255
        x_test /= 255

        y_train = keras.utils.to_categorical(y_train, num_classes)
        y_test = keras.utils.to_categorical(y_test, num_classes)
        
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        
        self.generator = ImageDataGenerator(
            rotation_range=15,
            rescale=1.0/255,
            shear_range=0.2,
            zoom_range=0.2)
        self.generator.fit(x_train, seed=randint(10, 100))


학습효과 분석

In [4]:
from utils.skeras import plot_loss, plot_acc
import matplotlib.pyplot as plt

# autoencoder 결과를 시각화
from keras import backend

def show_ae(ae, data):
    x_test = data.x_test
    decoded_imgs = ae.predict(x_test)
    print(decoded_imgs.shape, data.x_test.shape)
    
    if backend.image_data_format() == 'channels_first':
        N, n_ch, n_i, n_j = x_test.shape
    else:
        N, n_i, n_j, n_ch = x_test.shape
        
    x_test = x_test.reshape(N, n_i, n_j)
    decoded_imgs = decoded_imgs.reshape(decoded_imgs.shape[0], n_i, n_j)
    
    print("decoded_imgs.shape=", decoded_imgs.shape)
    
    n = 10
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i], cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i], cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        
    plt.show()

학습 및 확인

In [9]:
def main(epochs = 10, batch_size=128):
    data = DATA()
    ae = AE(data.input_shape)

    '''
    history = ae.fit(data.x_train, data.x_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_split=0.2)
    '''
    
    print(data.x_train.shape, data.y_train.shape)
    
    history = ae.fit_generator(
                data.generator.flow(data.x_train, data.y_train, batch_size=1000),
                epochs = 1,
                steps_per_epoch = 1000)
    
    plot_acc(history)
    plt.show()
    plot_loss(history)
    plt.show()
    
    show_ae(ae,data)
    plt.show()
    
if __name__ == '__main__':
    main()

(60000, 28, 28, 1) (60000, 10)
Epoch 1/1


ValueError: Error when checking target: expected conv2d_35 to have 4 dimensions, but got array with shape (1000, 10)